In [6]:
import sklearn
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split
mlp = MLPClassifier()

from environment.environment import Environment
from utils import save_pickle, load_pickle

import time

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
env = Environment(brick_ip='ev3dev.local', get_state_mode='dict')
env.get_state()

{'cs': ((151, 133, 215),)}

if on:
    calc_reward

In [40]:
class StateTrainer:
    def __init__(self, env, class_names, samples_per_class, clf, test_size=0.1):
        self.class_names = class_names
        self.class_map = {
            class_name: number for number, class_name in enumerate(self.class_names)
        }
        self.samples_per_class = samples_per_class
        self.clf = clf
        self.test_size = test_size
        
        self.env = env
        
        self.measurements = None
        self.X = None
        self.y = None
        
    def _rgb(self):
        return self.env.get_state()['cs'][0]
        
    def _button1_pressed(self):
        return self.env.get_state()['ts1'][0]
        
    def _gather_measurements(self):
        final_measurements = []
        for class_n, class_name in enumerate(self.class_names):
            class_measurements = []
            input("Press Enter to continue...")
            print(class_name)
            for i in range(self.samples_per_class):
                print("Collecting samples...")
                
#                 print("Press button 1 to collect a sample.")
#                 while not self._button1_pressed():
#                     pass

                time.sleep(0.3)
                colors1 = self._rgb()
                colors2 = self._rgb()
                colors = np.array([colors1, colors2])
                class_measurements.append(colors)
#                 print("Collected sample:", colors, "of class", class_name)
            final_measurements.append(class_measurements)
        return final_measurements

    def get_data_for_training(self):
        if self.measurements is None:
            self.measurements = self._gather_measurements()
        else:
            print("Already have measurements.")
        
        measurements = [np.array(measurement).reshape(-1, 6) for measurement in self.measurements]
        X = np.concatenate(measurements)
        y_ = [[self.class_map[class_name]]*self.samples_per_class for class_name in self.class_names]
        y = np.concatenate(y_)
        self.X = X
        self.y = y
        
        return train_test_split(X, y, test_size=self.test_size, stratify=y)
    
    def train(self)
        X_train, X_test, y_train, y_test = self.get_data_for_training()
        self.clf.fit(X_train, y_train)
        return self.clf.score(X_test, y_test)
    
    def save_model(self, name):
        save_pickle(name, self.clf)

In [41]:
class_names = ['off', 'on']
samples_per_class = 100
st = StateTrainer(env, class_names, samples_per_class=samples_per_class, clf=mlp, test_size=0.1)

In [42]:
print(st.train())
st.save_model('mlp_on_off.pickle')

Press Enter to continue...
off
Press Enter to continue...
on
1.0


In [43]:
mlp = load_pickle('mlp_on_off.pickle')

In [44]:
for i in range(5):
    while not button1_pressed():
        pass
    colors = st._rgb()
    print(mlp.predict(np.array(colors).reshape(1,-1)))

NameError: name 'button1_pressed' is not defined

In [109]:
meas_save = st.measurements

In [47]:
class_names = ['white', 'black']
samples_per_class = 50
st = StateTrainer(env, class_names, samples_per_class=samples_per_class, clf=LogisticRegressionCV(), test_size=0.1)
# st.measurements = meas_save
print(st.class_names, st.class_map)
print(st.train())
st.save_model('mlp_white_black.pickle')

['white', 'black'] {'white': 0, 'black': 1}
Press Enter to continue...
white
Press Enter to continue...
black


C:\Users\Michal\AppData\Roaming\Python\Python36\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


1.0


In [48]:
clf = load_pickle('mlp_white_black.pickle')

In [128]:
for i in range(5):
    while not button1_pressed():
        pass
    colors = st._rgb()
    print(clf.predict_proba(np.array(colors).reshape(1,-1)))

[[5.56272072e-11 7.62900265e-01 2.37099735e-01]]
[[1.54198023e-13 7.44306985e-01 2.55693015e-01]]
[[2.17656940e-14 6.72710183e-01 3.27289817e-01]]
[[3.86693673e-11 7.32640336e-01 2.67359664e-01]]
[[9.24001487e-29 2.10605892e-01 7.89394108e-01]]
